In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()

In [16]:
stoi = {s: i+1 for i, s in enumerate(sorted(list(set(''.join(words)))))}
stoi['.'] = 0
stoi

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '.': 0}

In [23]:
context_length = 3 
X, Y = [], []

for word in words[:5]:
    # initialize the context for the word: 
    context  = ['.'] * context_length
    for ch in word + '.':
        X.append([stoi[s] for s in context])
        Y.append(stoi[ch])
        print(context, '->', ch)
        context =  context[1:] + [ch]
        
    

['.', '.', '.'] -> e
['.', '.', 'e'] -> m
['.', 'e', 'm'] -> m
['e', 'm', 'm'] -> a
['m', 'm', 'a'] -> .
['.', '.', '.'] -> o
['.', '.', 'o'] -> l
['.', 'o', 'l'] -> i
['o', 'l', 'i'] -> v
['l', 'i', 'v'] -> i
['i', 'v', 'i'] -> a
['v', 'i', 'a'] -> .
['.', '.', '.'] -> a
['.', '.', 'a'] -> v
['.', 'a', 'v'] -> a
['a', 'v', 'a'] -> .
['.', '.', '.'] -> i
['.', '.', 'i'] -> s
['.', 'i', 's'] -> a
['i', 's', 'a'] -> b
['s', 'a', 'b'] -> e
['a', 'b', 'e'] -> l
['b', 'e', 'l'] -> l
['e', 'l', 'l'] -> a
['l', 'l', 'a'] -> .
['.', '.', '.'] -> s
['.', '.', 's'] -> o
['.', 's', 'o'] -> p
['s', 'o', 'p'] -> h
['o', 'p', 'h'] -> i
['p', 'h', 'i'] -> a
['h', 'i', 'a'] -> .


In [25]:
X = torch.tensor(X)
Y = torch.tensor(Y)
print(X)
print(Y)

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])
tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])


/var/folders/ry/1ct1kks556s6ffsdyjq1grdh0000gn/T/ipykernel_9588/4019588797.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X)
/var/folders/ry/1ct1kks556s6ffsdyjq1grdh0000gn/T/ipykernel_9588/4019588797.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(Y)


In [26]:
# embedding matrix: C: 27,2 
C = torch.rand(27,2)

In [28]:
C.shape

torch.Size([27, 2])

In [34]:
C[X[0]]

tensor([[0.1681, 0.9909],
        [0.1681, 0.9909],
        [0.1681, 0.9909]])

Embedding Structure:

- `C` is an embedding matrix of `(27, 2)`
- `X` is list  of context window, where each context is a list of indicies

When we do `C[X]` we get tensor shaped `(32, 3, 2)`, where:
- the first dimension (32) is the number of context windows
- the second dimension (3) is the number of characters in each context window
- the third dimension (2) is the dimensionality of the context window

We should view `emb` as 3-dimensional tensor where each "element" is a `3,2` of matrix

In [39]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [43]:
emb[0]

tensor([[0.1681, 0.9909],
        [0.1681, 0.9909],
        [0.1681, 0.9909]])

Neural Networks, particularly MLPs, expect input to be flat vectors. Our new vector represents the entire context window as a single unit of information, which the neural network can process.


In [44]:
emb.view(-1,6)[0]

tensor([0.1681, 0.9909, 0.1681, 0.9909, 0.1681, 0.9909])